# Molecular Simulation

Using Lennard-Jones potential

In [1]:
import numpy as np
import pygame
import math

pygame 2.6.0 (SDL 2.28.4, Python 3.12.6)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [37]:
MASS = 1.
SIGMA = 1e-2
EPSILON = 1

### Code stuff

In [38]:
# Force derived from Lennard-Jones potential, F = -gradV
def lennard_jones_force(X):
	'''
	X = [dpx, dpy, vx, vy], an ndarray
	dpx, dpy: difference between position of molecule that
	will be updated and molecule that is creating the force
	vx, vy: velocity of molecule that will be updated
	'''
	dpx, dpy, vx, vy = X
	Dpx = vx
	Dpy = vy
	drs = dpx**2 + dpy**2 + 1e-4 # Distance between particles squared
	c = 24 * EPSILON * (SIGMA ** 6) * (1 / MASS) * (drs ** (-4)) * (2 * (SIGMA ** 6) * (drs ** (-3)) - 1) # Insane constant
	Dvx = c * dpx
	Dvy = c * dpy
	return np.array([Dpx, Dpy, Dvx, Dvy])

In [39]:
def RK_method(F, X, h, a, b, c):
	dim = len(X)
	l = len(b)
	k = np.zeros((l, dim))

	k[0] = F(X)
	for i in range(1, l):
		x_i = X + h * np.dot(a[i-1][:i], k[:i])
		k[i] = F(x_i)
		
	return h * np.dot(b, k)

a = np.array(
    [[0.5, 0], [-1, 2]]
)
b = np.array([1/6, 4/6, 1/6])
c = np.array([0, 0.5, 1])

def RK3(F, X, h):
    return RK_method(F, X, h, a, b, c)

In [40]:
# Update particles according to the ODE that determines the system
def simulation_step(particles, h):
	# Copy particles array
	updated_particles = np.copy(particles)

	# Initialize grid which will store particle indices in cells
	# Particles will only interact with particles in neighboring cells
	grid = dict() # Store indices of particles in array with key being the idx of a cell
	reversed_grid = dict() # Store pairs particle idx : grid idx
	n = math.floor(1 / (2.5 * SIGMA)) # n^2 = number of cells
	l = 1 / n # side length of cell
	for i in range(-1, n+1): # This range will be used so that there isn't an out of bound error later
		for j in range(-1, n+1):
			grid[(i, j)] = [] # Indices will be stored in the array

	# Add particles to their respective cells, O(len(particles))
	for k, p in enumerate(particles):
		# Compute the cell the particle is in and add it to cell
		px, py = p[:2]
		i = round(n * (px - (px % l)))
		j = round(n * (py - (py % l)))
		grid[(i, j)].append(k)
		reversed_grid[k] = (i, j)

	# Main loop
	for k, p1 in enumerate(particles):
		# Get cell index of p1
		i, j = reversed_grid[k]

		# List cells p1 can interact with
		# There won't be bound errors, read comment above
		cells = [
			(i-1, j-1), (i-1, j), (i-1, j+1),
         	(i, j-1),   (i, j),   (i, j+1),
         	(i+1, j-1), (i+1, j), (i+1, j+1),
		]

		for cell in cells:
			for p2_idx in grid[cell]:
				p2 = particles[p2_idx]
				if p1 is not p2:
					# Compute interactions
					dpx, dpy = p1[:2] - p2[:2]
					vx, vy = p1[2:]
					X = np.array([dpx, dpy, vx, vy])
					updated_particles[k] += RK3(lennard_jones_force, X, h)

		# Deal with boundary conditions
		if updated_particles[k, 0] < 0:
			updated_particles[k, 0] = 1e-6
			updated_particles[k, 2] *= -1
		elif updated_particles[k, 0] >= 1:
			updated_particles[k, 0] = 1 - 1e-6
			updated_particles[k, 2] *= -1
		if updated_particles[k, 1] < 0:
			updated_particles[k, 1] = 1e-6
			updated_particles[k, 3] *= -1
		elif updated_particles[k, 1] >= 1:
			updated_particles[k, 1] = 1 - 1e-6
			updated_particles[k, 3] *= -1
			
	return updated_particles

In [41]:
# Initialize pygame display
def init_pygame_display(window_size=(500, 500)):
    pygame.init()
    screen = pygame.display.set_mode(window_size)
    pygame.display.set_caption("Position Simulation")
    return screen

# Function to display simulation using pygame
def run_simulation(particles, window_size=(500, 500), point_radius=2, speed=1e-2):
    screen = init_pygame_display(window_size)
    clock = pygame.time.Clock()
    running = True
    
    while running:
        # Check for Pygame events (such as quitting the simulation)
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
                pygame.quit()
                return  # Exit function
        
        # Update particles' positions with the simulation step
        particles = simulation_step(particles, speed)
        
        # Scale the particles' positions to fit within the window
        scaled_particles = particles[:, :2] * window_size[0]  # Assuming positions are in [0, 1]
        
        # Clear the screen with a white background
        screen.fill((255, 255, 255))

        # Draw each particle as a circle
        for pos in scaled_particles:
            x, y = pos
            pygame.draw.circle(screen, (0, 0, 255), (int(x), int(y)), point_radius)

        # Update the display
        pygame.display.flip()

        # Cap the frame rate at 30 FPS
        clock.tick(30)

### Actual sim

In [46]:
n_particles = 1000
h = 1e-4

# Each particle has an px, py position and vx, vy velocity
particles = np.random.uniform(size=(n_particles, 4))
particles[:, 2:] = 10 * (particles[:, 2:] - 0.5)

for _ in range(1): # each step takes around 4e-2 seconds
	particles = simulation_step(particles, h)

In [47]:
run_simulation(particles, window_size=(500, 500), point_radius=2, speed=h)